完成从 Xsens txt 文件到 OpenSim .sto 文件转换

In [4]:
# 查看包含四元数的 .sto 数据
import os
filename = "/Users/wangshuaibo/Library/CloudStorage/OneDrive-bsu.edu.cn/Archive/代码_OpenSense/OpenSense_Python/OpenSenseExample/OpenSenseExample_Modified/MT_012005D6_009-001_orientations.sto"

with open(filename, "r") as f:
    count = 0
    for line in f:
        count +=1
        if "endheader" in line:
            metadata_row_num = count

    metadata = []   
    count = 0
    for line in f:
        count += 1
        if count<metadata_row_num:
            print(line)
            metadata.append(line)
        
        if count==7:
            test_data = line
            break
    
total_cout = count
print(f"文件共包含 {total_cout} 行")
print(f"头文件包含在前 {metadata_row_num} 行")
print(f"头文件为：{metadata}")

# 验证 sto 中的数据类型
print(f".sto中的内容同样为字符串{type(test_data[0])}")
# 查看 sto 中的数据格式
print(f".sto 的格式为:\n{repr(test_data)}")


文件共包含 0 行
头文件包含在前 5 行
头文件为：[]


NameError: name 'test_data' is not defined

In [43]:
# 定义演示数据
import pandas as pd
columns = ["time"] + [segment + quat for segment in ['torso', 'pelvis', 'tibia_r'] for quat in ["_q0","_q1","_q2","_q3"]]
df = pd.DataFrame([[0,   1,0,0,0,1,0,0,0,1,0,0,0],
                   [0.01,1,0,0,0,1,0,0,0,1,0,0,0]],
                  columns=columns)


In [44]:
# 转换为 opensim 格式
line_len = (df.shape[1]+1) * 2

def line_df_to_sto(line_len,row=0):
    line = [None] * line_len
    for i in range(line_len):
        # 此处也需要修改
        line[0] = df.iloc[row,0]
        if i%2 != 0 and (i-1)%8 != 0:
            line[i] = ","
        elif (i-1)%8 == 0:
            line[i] = "\t"
        elif i != 0 :
            #print(i)
            line[i] = df.iloc[row,int(i/2-1)]

    # 将列表转换为字符串
    result = ""
    for n in line:
        result += str(n)
    return result.strip("\t") + "\n"


In [53]:
line_df_to_sto(line_len,row=0)

'0.0\t0.0,1,0,0\t0,1,0,0\t0,1,0,0\t0,\n'

In [49]:
harzt = 100
# 将hartz 填入表头
metadata = f'''DataRate={harzt:.6f}
DataType=Quaternion
version=3
OpenSimVersion=4.4-2022-10-11-798caa8
endheader\n'''

header_list = ["time"] + [segment+"_imu" for segment in ['torso', 'pelvis', 'tibia_r']]
header_str = "\t".join(header_list) + "\n"


In [51]:
with open("test_quat.sto", "w") as f:
    # 先写属性信息
    f.write(metadata)
    f.write(header_str)
    for j in range(df.shape[0]):
        sto_line = line_df_to_sto(line_len,row=j)
        f.write(sto_line)
    

In [47]:
## 定义关节部位及其对应的传感器编号

In [48]:
segments = ['torso',   'pelvis',   'tibia_r',  'femur_r', 'tibia_l', 'femur_l', 'calcn_r', 'calcn_l']
sensors = ["_00B42268","_00B42279","_00B4227D","_00B4227C","_00B421ED","_00B421EE","_00B421EF","_00B421E6"]
IMUMapping = dict(zip(segments,sensors))
print("解剖位置与传感器部位的对应关系为：\n",IMUMapping)

解剖位置与传感器部位的对应关系为：
 {'torso': '_00B42268', 'pelvis': '_00B42279', 'tibia_r': '_00B4227D', 'femur_r': '_00B4227C', 'tibia_l': '_00B421ED', 'femur_l': '_00B421EE', 'calcn_r': '_00B421EF', 'calcn_l': '_00B421E6'}


## 将所有的传感器四元数数据合并到一个文件夹

In [25]:
path = "/Users/wangshuaibo/Library/CloudStorage/OneDrive-bsu.edu.cn/Archive/代码_OpenSense/OpenSense_Python/OpenSenseExample/OpenSenseExample_Original_20230518"
print(f"当前文件夹下数据：\n{os.listdir(path)}")

当前文件夹下数据：
['myIMUIK_Setup.xml', 'myIMUMappings.xml', '.DS_Store', 'Rajagopal_2015.osim', 'OpenSense_IMUDataConverter.py', 'OpenSense_OrientationTracking.py', 'Geometry', 'OpenSense_CalibrateModel.py', 'IMUData', 'myIMUPlacer_Setup.xml']


In [27]:
IMU_path = os.path.join(path,"IMUData")
os.listdir(IMU_path)

['MT_012005D6_009-001_00B4227D.txt',
 'MT_012005D6_009-001_00B4227C.txt',
 'MT_012005D6_009-001_00B421E6.txt',
 'MT_012005D6_009-001_00B42279.txt',
 'MT_012005D6_009-001_00B421EF.txt',
 'MT_012005D6_009-001_00B42268.txt',
 'MT_012005D6_009-001_00B421EE.txt',
 'MT_012005D6_009-001_00B421ED.txt']

In [33]:
data = pd.read_csv(os.path.join(IMU_path, 'MT_012005D6_009-001_00B421ED.txt'),skiprows=5,sep="\t")
data

,PacketCounter,SampleTimeFine,Year,Month,Day,Second,UTC_Nano,UTC_Year,UTC_Month,UTC_Day,...,Acc_Z,Mat[1][1],Mat[2][1],Mat[3][1],Mat[1][2],Mat[2][2],Mat[3][2],Mat[1][3],Mat[2][3],Mat[3][3]
0,472,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.970936,-0.102187,-0.059505,0.992984,0.068831,0.995394,0.066732,-0.992381,0.075167,-0.097620
1,473,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.943480,-0.102192,-0.059407,0.992989,0.068608,0.995417,0.066613,-0.992396,0.074934,-0.097648
2,474,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.946427,-0.102104,-0.059314,0.993004,0.068465,0.995435,0.066499,-0.992415,0.074776,-0.097577
3,475,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.916622,-0.102045,-0.059230,0.993015,0.068183,0.995462,0.066383,-0.992440,0.074481,-0.097543
4,476,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.966246,-0.101912,-0.059215,0.993029,0.067825,0.995490,0.066323,-0.992479,0.074112,-0.097436
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2476,2948,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-2.176041,-0.581143,0.369453,0.725105,0.567231,-0.455012,0.686450,0.583542,0.810228,0.054862
2477,2949,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-1.270531,-0.604637,0.368032,0.706376,0.563159,-0.429622,0.705887,0.563264,0.824607,0.052505
2478,2950,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.581064,-0.630645,0.363574,0.685639,0.560143,-0.398242,0.726391,0.537147,0.842150,0.047496
2479,2951,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.221149,-0.656325,0.357613,0.664342,0.557502,-0.363421,0.746402,0.508359,0.860255,0.039152


In [38]:
import numpy as np
from scipy.spatial.transform import Rotation as R


matrix = ['Mat[1][1]','Mat[1][2]','Mat[1][3]',
          'Mat[2][1]','Mat[2][2]','Mat[2][3]',
          'Mat[3][1]','Mat[3][2]','Mat[3][3]']
matrix_array = [['Mat[1][1]','Mat[1][2]','Mat[1][3]'],
          ['Mat[2][1]','Mat[2][2]','Mat[2][3]'],
          ['Mat[3][1]','Mat[3][2]','Mat[3][3]']]
def XsensMat2Quat(matrix):
    r = R.from_matrix(matrix)
    cal_quat = -r.as_quat()
    quat_wxyz = cal_quat
    return -np.array(quat_wxyz,6)

data["qw","qx","qy","qz"] = data[matrix].apply(XsensMat2Quat)


ValueError: Expected `matrix` to have shape (3, 3) or (N, 3, 3), got (2481,)

In [54]:
%pprint pd.read_csv

Pretty printing has been turned OFF


In [55]:
import pandas as pd

read_csv = pd.read_csv

print(read_csv.__code__)

<code object wrapper at 0x7f9ab84c3aa0, file "/Users/wangshuaibo/anaconda3/lib/python3.10/site-packages/pandas/util/_decorators.py", line 170>


@deprecate_kwarg(old_arg_name="mangle_dupe_cols", new_arg_name=None)
@deprecate_nonkeyword_arguments(version=None, allowed_args=["filepath_or_buffer"])
@Appender(
    _doc_read_csv_and_table.format(
        func_name="read_csv",
        summary="Read a comma-separated values (csv) file into DataFrame.",
        _default_sep="','",
        storage_options=_shared_docs["storage_options"],
        decompression_options=_shared_docs["decompression_options"]
        % "filepath_or_buffer",
    )
)
def read_csv(
    filepath_or_buffer: FilePath | ReadCsvBuffer[bytes] | ReadCsvBuffer[str],
    sep: str | None | lib.NoDefault = lib.no_default,
    delimiter: str | None | lib.NoDefault = None,
    # Column and Index Locations and Names
    header: int | Sequence[int] | None | Literal["infer"] = "infer",
    names: Sequence[Hashable] | None | lib.NoDefault = lib.no_default,
    index_col: IndexLabel | Literal[False] | None = None,
    usecols=None,
    squeeze: bool | None = None,
    prefix: st